In [1]:
import pandas as pd
import yfinance as yf
stk_data = yf.download("RELIANCE.NS", start="2021-06-01", end="2022-05-01")
print(stk_data.head())

C:\Users\tiruk\AppData\Local\Temp\ipykernel_14520\620575181.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stk_data = yf.download("RELIANCE.NS", start="2021-06-01", end="2022-05-01")
[*********************100%***********************]  1 of 1 completed

Price             Close         High         Low         Open      Volume
Ticker      RELIANCE.NS  RELIANCE.NS RELIANCE.NS  RELIANCE.NS RELIANCE.NS
Date                                                                     
2021-06-01   987.374939  1002.443448  977.177463   986.054707    28013832
2021-06-02  1002.101929  1005.857676  981.957442   981.957442    24630162
2021-06-03  1005.925964  1024.294990  999.757459  1011.092917    23974257
2021-06-04   997.208069  1009.021575  994.362780  1008.270425    14468646
2021-06-07  1014.006409  1020.652962  994.704167  1002.443306    16336573


In [2]:
dataset = pd.read_csv("reliance_data.csv", skiprows=1)
import pandas as pd
# Flatten column MultiIndex if exists
if isinstance(stk_data.columns, pd.MultiIndex):
    stk_data.columns = [col[0] for col in stk_data.columns]

# Check data
print("Data fetched successfully!")
print(stk_data.head())

Data fetched successfully!
                  Close         High         Low         Open    Volume
Date                                                                   
2021-06-01   987.374939  1002.443448  977.177463   986.054707  28013832
2021-06-02  1002.101929  1005.857676  981.957442   981.957442  24630162
2021-06-03  1005.925964  1024.294990  999.757459  1011.092917  23974257
2021-06-04   997.208069  1009.021575  994.362780  1008.270425  14468646
2021-06-07  1014.006409  1020.652962  994.704167  1002.443306  16336573


In [3]:
stk_data=stk_data[["Open","High","Low","Close"]]
stk_data

,Open,High,Low,Close
Date,,,,
2021-06-01,986.054707,1002.443448,977.177463,987.374939
2021-06-02,981.957442,1005.857676,981.957442,1002.101929
2021-06-03,1011.092917,1024.294990,999.757459,1005.925964
2021-06-04,1008.270425,1009.021575,994.362780,997.208069
2021-06-07,1002.443306,1020.652962,994.704167,1014.006409
...,...,...,...,...
2022-04-25,1252.808769,1260.117952,1216.833432,1231.155029
2022-04-26,1238.007307,1276.837847,1236.956647,1267.998169
2022-04-27,1258.952988,1291.913249,1258.587463,1269.231689


In [4]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (228, 4)


In [5]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [6]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

182
X_train length: (182, 4)
X_test length: (46, 4)
y_train length: (182, 4)
y_test length: (46, 4)


In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [9]:
def cominbation(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [10]:
listt=["Close","High","Open","Low"]

In [11]:
perf,result,pred=cominbation(data1,listt)

['Close', 'High', 'Open', 'Low']
Order = 1
AIC:  -30.456225115004134
BIC:  -30.125239705484084

Order = 2
AIC:  -30.42108992042171
BIC:  -29.823223187568157

Order = 3
AIC:  -30.38176935677695
BIC:  -29.515136900460323

Order = 4
AIC:  -30.345264209399897
BIC:  -29.20795912354441

Order = 5
AIC:  -30.221295669245055
BIC:  -28.81138816709452

Order = 6
AIC:  -30.211946306107176
BIC:  -28.527483337517832

Order = 7
AIC:  -30.173671264866446
BIC:  -28.212676125421023

Order = 8
AIC:  -30.145018311064057
BIC:  -27.905490242794958

Order = 9
AIC:  -30.176837963139697
BIC:  -27.656751746642833

Order = 10
AIC:  -30.071234779126762
BIC:  -27.26854031684087



In [12]:
data1

,Open,High,Low,Close
0,0.130244,0.170833,0.159502,0.160685
1,0.119063,0.180197,0.173094,0.201775
2,0.198571,0.230763,0.223707,0.212445
3,0.190869,0.188874,0.208367,0.188120
4,0.174967,0.220774,0.209338,0.234990
...,...,...,...,...
223,0.858194,0.877529,0.840943,0.840864
224,0.817802,0.923385,0.898162,0.943662
225,0.874961,0.964731,0.959667,0.947103
226,0.942405,0.993548,1.000000,1.000000


In [13]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0,0.283124,1,28
